In [1]:
import pandas as pd

In [2]:
superfund_df = pd.read_csv('resources/superfund_sites.csv')

In [3]:
superfund['address'] = 

,RG,SITE_ID,SITE_EPA_ID,SITE_NAME,SITE_STRT_ADRS1,SITE_STRT_ADRS2,SITE_CITY_NAME,SITE_STATE,SITE_ZIP_CODE,SITE_CONG_DISTRICT,SITE_CNTY_NAME,SITE_FIPS_CODE,LATITUDE,LONGITUDE,ALT_AGREEMENT,NPL,FF,NON_NPL_STATUS
0,4,406154,AL0000047829,WEST END LANDFILL,HWY 202 AND ADAMS ST,NaN,ANNISTON,AL,36202,03,CALHOUN,1015.0,NaN,NaN,N,Not on the NPL,N,N
1,4,406158,AL0000059147,ETOWAH COUNTY (RIDGEVILLE) LANDFILL,NOBLE HILL ROAD,NaN,RIDGEVILLE,AL,35954,04,ETOWAH,1055.0,NaN,NaN,N,Not on the NPL,N,N
2,4,406176,AL0000132662,FUSSY HILL DRUM DUMP,461 FUSSY HILL ROAD,NaN,NEW MARKET,AL,35761,05,MADISON,1089.0,NaN,NaN,N,Not on the NPL,N,N
3,4,406800,AL0000301747,JONES PROPERTY,595 GRADY STREET,NaN,MONTGOMERY,AL,36108,NaN,MONTGOMERY,1101.0,NaN,NaN,N,Not on the NPL,N,N
4,4,406321,AL0001024793,ILCO SATELLITE SITES,INTERSECTION OF HWY 78 AND HWY 411,NaN,LEEDS,AL,35094,06,JEFFERSON,1073.0,NaN,NaN,N,Not on the NPL,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13162,8,820977,WYN000820977,HARTVILLE (TRONOX),Hartville Hwy 270 near the town of Hartville,NaN,HARTVILLE,WY,82215,01,PLATTE,56031.0,NaN,NaN,N,Not on the NPL,N,N
13163,8,820979,WYN000820979,PETROTOMICS MILL (TRONOX),"Intersection of Wyoming 487 and Shriley Ridge Rd,",approximately 30 miles north of Medicine Bow,MEDICINE BOW,WY,82329,01,CARBON,56007.0,NaN,NaN,N,Not on the NPL,N,N
13164,8,820999,WYN000820999,THOMES AVENUE MERCURY SPILL,Intersection of W. 5th Street and Thomes Avenue,NaN,CHEYENNE,WY,82007,01,LARAMIE,56021.0,NaN,NaN,N,Not on the NPL,N,N
13165,8,821050,WYN000821050,YELLOWSTONE NATIONAL PARK GASOLINE SPILL,Yellowstone National Park,NaN,TOWER JUNCTION,WY,82190,00,PARK,56029.0,NaN,NaN,N,Not on the NPL,N,N
